In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
train_generator = train_datagen.flow_from_directory(
        '../data/archive/train/mouth-cropped',  # This is the source directory for training images
        target_size=(80, 80),  # All images will be resized to 150x150
        # Since we use binary_crossentropy loss, we need binary labels
        shuffle=True,
        class_mode='binary',
        batch_size=16)

validation_generator = train_datagen.flow_from_directory(
        '../data/archive/val/mouth-cropped',  # This is the source directory for training images
        target_size=(80, 80),  # All images will be resized to 150x150
        # Since we use binary_crossentropy loss, we need binary labels
        shuffle=True,
        class_mode='binary',
        batch_size=16)

Found 856 images belonging to 2 classes.
Found 266 images belonging to 2 classes.


In [5]:
base_model = tf.keras.applications.InceptionResNetV2(
    include_top=False, weights='imagenet', input_shape=(80,80,3)
)

In [24]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [25]:
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 1, 1, 1536)       54336736  
 onal)                                                           
                                                                 
 global_average_pooling2d_1   (None, 1536)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_1 (Dense)             (None, 1)                 1537      
                                                                 
Total params: 54,338,273
Trainable params: 54,277,729
Non-trainable params: 60,544
_________________________________________________________________


In [27]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

history = model.fit(
      train_generator,
      validation_data = validation_generator,
      epochs=50,
      verbose=1,
      callbacks=early_stop)

Epoch 1/50
54/54 [==============================] - 53s 424ms/step - loss: 0.0732 - accuracy: 0.9743 - val_loss: 1.1273 - val_accuracy: 0.7030
Epoch 2/50
54/54 [==============================] - 19s 344ms/step - loss: 0.0563 - accuracy: 0.9825 - val_loss: 0.4194 - val_accuracy: 0.8609
Epoch 3/50
54/54 [==============================] - 19s 344ms/step - loss: 0.0508 - accuracy: 0.9848 - val_loss: 0.2147 - val_accuracy: 0.9211
Epoch 4/50
54/54 [==============================] - 18s 343ms/step - loss: 0.0291 - accuracy: 0.9895 - val_loss: 7.1601 - val_accuracy: 0.7030
Epoch 5/50
54/54 [==============================] - 18s 335ms/step - loss: 0.0253 - accuracy: 0.9930 - val_loss: 1.6562 - val_accuracy: 0.9436
Epoch 6/50
54/54 [==============================] - 18s 333ms/step - loss: 0.0248 - accuracy: 0.9918 - val_loss: 0.1089 - val_accuracy: 0.9812
Epoch 7/50
54/54 [==============================] - 18s 331ms/step - loss: 0.0094 - accuracy: 0.9965 - val_loss: 0.1261 - val_accuracy: 0.9774

In [28]:
test_generator = train_datagen.flow_from_directory(
        '../data/test/yourfolderforcroppedmouths',  # This is the source directory for training images
        target_size=(80, 80),  # All images will be resized to 150x150
        # Since we use binary_crossentropy loss, we need binary labels
        shuffle=True,
        class_mode='binary',
        batch_size=16)

Found 46 images belonging to 2 classes.


In [29]:
model.evaluate(test_generator)

3/3 [==============================] - 1s 439ms/step - loss: 0.4835 - accuracy: 0.8696


[0.48349910974502563, 0.8695651888847351]

In [30]:
model.save("inception_cropped_yawning_model.h5")